# Basic Imports

In [1]:
from google.colab import drive
import sys
import os
import pandas as pd
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


#Linking the dataset to data in the sql dataset

This notebook is thinked as help to link every ingredient of the sql dataset by Amoruso & Fusillo to an entry of the Complete and Coherent Sueatable dataset (CSEL).

An embedding based algorithm for compute semantic similarity between strings will be used to catch the most relatable ingredient for each entry of the sql dataset given as csv file.

All the matches are given as suggestion, is necessary to manually check their reliability.

###Ingredient-name matching algorithm.

Search for match between the ingredient name in the FoodPrintDB and the food item name in the CSEL Dataset

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
ingredients = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/food_print_ingredients.csv")
sueatable = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/_Finalized/cfp_wfp_ingredients.csv",sep= ';')

embedding_sue = model.encode(sueatable['Food commodity ITEM'])

for _, ingredient_row in ingredients.iterrows():
  embedding_ing = model.encode(ingredient_row['name'])
  scores = []  
  scores = util.pytorch_cos_sim(embedding_ing, embedding_sue).flatten()
  sorted = scores.argsort()[-2:]
  print(ingredient_row['name'])
  for i in sorted:
    max_score_idx = int(i)
    max_score = float(scores[max_score_idx])
    
    print(sueatable['Food commodity ITEM'][max_score_idx])
    print(max_score)
    print("update ingredients set carbon_foot_print ="+str(sueatable['final_co2'][max_score_idx]).replace(",", ".")+", water_foot_print = "+str(sueatable['final_wfp'][max_score_idx]).replace(",", ".")+", carbon_foot_print_source	= 'sueatable', carbon_foot_print_weight=1000, water_foot_print_source= 'sueatable', water_foot_print_weight =	1000 where ingredient_id ="+str(ingredient_row['ingredient_id'])+";")
    print("insert into ingredients_name_alias values ("+str(ingredient_row['ingredient_id'])+",'"+str(sueatable['Food commodity ITEM'][max_score_idx])+"');")
    print('***************************************')
  print('#######################################')

###Ingredient-typology matching algorithm.

Search for match between the ingredient name in the FoodPrintDB and the cfp/wf typology of the SEL Database.

Could be useful applyu a filter on the FoodPrintDB csv in order to work only on prfeviosly missed ingredients (in the next section there is a working example of filtering on FoodPrintDB).

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
ingredients = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/food_print_ingredients.csv")

co2_typ = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/co2_median_typology.csv", sep= ';')
wfp_typ = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/wfp_median_typology.csv", sep= ';')

#remove from the "Typology" fields * and parentheses
prt_regex = '\((\w*\W*\s*)*\)' #remove parentheses and their content
star_regex = '\*' #remove *

co2_typ['FOOD COMMODITY TYPOLOGY'] = co2_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True)
co2_typ['FOOD COMMODITY TYPOLOGY'] = co2_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True)

wfp_typ['FOOD COMMODITY TYPOLOGY'] = wfp_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True)
wfp_typ['FOOD COMMODITY TYPOLOGY'] = wfp_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True)

###############################################################################


embedding_co2_typ = model.encode(co2_typ['FOOD COMMODITY TYPOLOGY'], convert_to_tensor=True)
embedding_wfp_typ = model.encode(wfp_typ['FOOD COMMODITY TYPOLOGY'], convert_to_tensor=True)


for _, ingredient_row in ingredients.iterrows():
  embedding_ing = model.encode(ingredient_row['name'], convert_to_tensor=True)

  scores_co2_typ = []
  scores_wfp_typ = []  
  scores_co2_typ = util.pytorch_cos_sim(embedding_ing, embedding_co2_typ)  
  scores_wfp_typ = util.pytorch_cos_sim(embedding_ing, embedding_wfp_typ)  
  
  #select the right co2 typology
  co2_max_score_idx = int(np.argmax(scores_co2_typ))
  co2_max_score = float(scores_co2_typ[0,co2_max_score_idx])
  #select the right wfp typology
  wfp_max_score_idx = int(np.argmax(scores_wfp_typ))
  wfp_max_score = float(scores_wfp_typ[0,wfp_max_score_idx])

  print(ingredient_row['name'])
  print('****Guessing typologies for cfp and wfp****')
  guessed_typology = co2_typ['FOOD COMMODITY TYPOLOGY'][co2_max_score_idx]
  print(guessed_typology)
  print(co2_max_score)
  print(wfp_typ['FOOD COMMODITY TYPOLOGY'][wfp_max_score_idx])
  print(wfp_max_score)
  print("update ingredients set carbon_foot_print ="+str(co2_typ['median'][co2_max_score_idx]).replace(",", ".")+", water_foot_print = "+str(wfp_typ['median'][wfp_max_score_idx]).replace(",", ".")+", carbon_foot_print_source	= 'sueatable_typology_median_values', carbon_foot_print_weight=1000, water_foot_print_source= 'sueatable_typology_median_values', water_foot_print_weight =	1000 where ingredient_id ="+str(ingredient_row['ingredient_id'])+";")
  print('#######################################')


###Place for finetune typology matching algorithm.

Used to force match between specific igrdiendts and cfp/wfp typologies.

Must be adapted to every specific circumstance.

In [5]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
ingredients = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/food_print_ingredients.csv")

d={'name': 
["nuts, pine nuts, dried",
"nuts, macadamia nuts, raw",
"nuts, pecans",
"kashi, steam meal, chicken fettuccine, frozen entree",
"dill weed, fresh",
"parsley, fresh",
"terra dolce, organic chipotle dried chile peppers, upc: 700360007144",
"spices, pepper, red or cayenne",
"spices, pepper, black",
"spices, basil, dried",
"kale, raw",
"chives, raw",
"squash, winter, butternut, raw",
"arugula, raw"],
'typology': 
["NUTS",
"NUTS",
"NUTS",
"POULTRY BONE FREE MEAT",
"SPICIES",
"SPICIES",
"SPICIES",
"SPICIES",
"SPICIES",
"SPICIES",
"VEGETABLES",
"VEGETABLES",
"VEGETABLES",
"VEGETABLES"]}

to_be_fixed = pd.DataFrame(data=d)

ingredients = pd.merge(ingredients, to_be_fixed, on='name', suffixes=('', '_new'))
print(ingredients['typology'].unique())

['NUTS' 'SPICIES' 'VEGETABLES' 'POULTRY BONE FREE MEAT']


In [6]:
co2_typ = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/co2_median_typology.csv", sep= ';')
wfp_typ = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/SuEatableLife/wfp_median_typology.csv", sep= ';')

#remove from the "Typology" fields * and parentheses
prt_regex = '\((\w*\W*\s*)*\)' #remove parentheses and their content
star_regex = '\*' #remove *

co2_typ['FOOD COMMODITY TYPOLOGY'] = co2_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True)
co2_typ['FOOD COMMODITY TYPOLOGY'] = co2_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace='VEGETABLES OPENFIELD', value='VEGETABLES', regex=True)
co2_typ['FOOD COMMODITY TYPOLOGY'] = co2_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True)

wfp_typ['FOOD COMMODITY TYPOLOGY'] = wfp_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace=prt_regex, value='', regex=True)
wfp_typ['FOOD COMMODITY TYPOLOGY'] = wfp_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace='NUTS WITH SHELL', value='', regex=True)
wfp_typ['FOOD COMMODITY TYPOLOGY'] = wfp_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace='NUTS SHELLED', value='NUTS', regex=True)
wfp_typ['FOOD COMMODITY TYPOLOGY'] = wfp_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace='LEGUMES FRESH', value='LEGUMES', regex=True)
wfp_typ['FOOD COMMODITY TYPOLOGY'] = wfp_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace='COCOA DERIVATES', value='CHOCOLATE', regex=True)
wfp_typ['FOOD COMMODITY TYPOLOGY'] = wfp_typ['FOOD COMMODITY TYPOLOGY'].replace(to_replace=star_regex, value='', regex=True)

###############################################################################
embedding_co2_typ = model.encode(co2_typ['FOOD COMMODITY TYPOLOGY'], convert_to_tensor=True)
embedding_wfp_typ = model.encode(wfp_typ['FOOD COMMODITY TYPOLOGY'], convert_to_tensor=True)

for _, ingredient_row in ingredients.iterrows():
  embedding_ing = model.encode(ingredient_row['typology'], convert_to_tensor=True)

  scores_co2_typ = []
  scores_wfp_typ = []  
  scores_co2_typ = util.pytorch_cos_sim(embedding_ing, embedding_co2_typ)  
  scores_wfp_typ = util.pytorch_cos_sim(embedding_ing, embedding_wfp_typ)  
  
  #select the right co2 typology
  co2_max_score_idx = int(np.argmax(scores_co2_typ))
  co2_max_score = float(scores_co2_typ[0,co2_max_score_idx])
  #select the right wfp typology
  wfp_max_score_idx = int(np.argmax(scores_wfp_typ))
  wfp_max_score = float(scores_wfp_typ[0,wfp_max_score_idx])

  print(ingredient_row['typology'])
  alias_name = ingredient_row['name'].upper().replace(',','')

  guessed_typology = co2_typ['FOOD COMMODITY TYPOLOGY'][co2_max_score_idx]
  print(guessed_typology)
  print(co2_max_score)
  print(wfp_typ['FOOD COMMODITY TYPOLOGY'][wfp_max_score_idx])
  print(wfp_max_score)
  print("update ingredients set carbon_foot_print ="+str(co2_typ['median'][co2_max_score_idx]).replace(",", ".")+", water_foot_print = "+str(wfp_typ['median'][wfp_max_score_idx]).replace(",", ".")+", carbon_foot_print_source	= 'sueatable_typology_median_values', carbon_foot_print_weight=1000, water_foot_print_source= 'sueatable_typology_median_values', water_foot_print_weight =	1000 where ingredient_id ="+str(ingredient_row['ingredient_id'])+";")
  print("insert into ingredients_name_alias values ("+str(ingredient_row['ingredient_id'])+",'"+alias_name+"');")
  print('#######################################')




NUTS
NUTS
1.0
NUTS
1.0
update ingredients set carbon_foot_print =1.11, water_foot_print = 11264, carbon_foot_print_source	= 'sueatable_typology_median_values', carbon_foot_print_weight=1000, water_foot_print_source= 'sueatable_typology_median_values', water_foot_print_weight =	1000 where ingredient_id =42;
insert into ingredients_name_alias values (42,'NUTS PINE NUTS DRIED');
#######################################
SPICIES
SPICIES
1.0
SPICIES
0.9999999403953552
update ingredients set carbon_foot_print =0.84, water_foot_print = 8280, carbon_foot_print_source	= 'sueatable_typology_median_values', carbon_foot_print_weight=1000, water_foot_print_source= 'sueatable_typology_median_values', water_foot_print_weight =	1000 where ingredient_id =61;
insert into ingredients_name_alias values (61,'SPICES PEPPER RED OR CAYENNE');
#######################################
NUTS
NUTS
1.0
NUTS
1.0
update ingredients set carbon_foot_print =1.11, water_foot_print = 11264, carbon_foot_print_source	= 'sueata